# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.

%load_ext dotenv
%dotenv

# %reload_ext dotenv

In [ ]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

# Write your code below.

directory_path = "../../05_src/data/prices"

PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(directory_path, "**", "*.parquet"), recursive=True)
len(parquet_files)


In [ ]:
for f in parquet_files:
    print(f)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)


In [ ]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker") #load data from parquet files to dask dataframe
dd_px.head()

In [ ]:
import os
import sys
import numpy as np
import dask.dataframe as dd

sys.path.append(os.getenv('SRC_DIR', ''))

# Import statements for logging 
from logger import get_logger
_logs = get_logger("DSI" + __name__)
_logs.info(dd_px.columns)


# Repartitioning the dataframe to increase partition size 
dd_px = dd_px.repartition(npartitions=1)

# Group by 'ticker' and calculate lags, returns, and range
dd_feat = (
    dd_px.groupby('ticker', group_keys=False)
    .apply(lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close_lag_1 = x['Adj Close'].shift(1),
        returns = (x['Adj Close'] / x['Adj_Close_lag_1']) - 1,
        hi_lo_range = x['High'] - x['Low']
    ))
)

# Print the first few rows of dd_feat for verification
print(dd_feat.head())




+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [ ]:
import pandas as pd
df = dd_feat.compute()
_logs.info(type(df))

df['Rolling_Average'] = df['returns'].rolling(10).mean()
df['Rolling_Average'] 


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

#You don't need to switch to pandas for the moving average return calculation. Given the data's size, using Dask could be more efficient. Dask allows you to handle data in smaller parts and process them simultaneously. Usually, it's better to do the calculation directly in Dask, especially with large datasets, because it scales well, works efficiently, and can process tasks in parallel.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.